In [1]:
using FastAI
using FluxTraining

In [2]:
dataset = Datasets.loaddataset("imagenette2-160");

method = ImageClassification(Datasets.loadclasses("imagenette2-160"), (160, 160))

dls = methoddataloaders(dataset, method)

model = methodmodel(method, Models.xresnet18());

learner = Learner(model, dls, ADAM(), methodlossfn(method), ToGPU(), Metrics(SmoothLoss(), accuracy))

Learner()

In [3]:
fit!(learner, ValidationPhase())

Epoch 1 ValidationPhase(): 100%|████████████████████████| Time: 0:00:39


┌─────────────────┬───────┬─────────┬────────────┬──────────┐
│           Phase │ Epoch │    Loss │ SmoothLoss │ Accuracy │
├─────────────────┼───────┼─────────┼────────────┼──────────┤
│ ValidationPhase │   1.0 │ 5.85771 │    6.40924 │  0.10231 │
└─────────────────┴───────┴─────────┴────────────┴──────────┘


Learner()

In [7]:
phase = FastAI.LRFinderPhase(steps = 100, β = 0.08)
fit!(learner, phase)

Epoch 1 FastAI.LRFinderPhase(1.0e-7, 10, 100, 0.08, 4, nothing):  13%  ETA: 0:13:33m

┌──────────────────────┬───────┬─────────┬────────────┐
│                Phase │ Epoch │    Loss │ SmoothLoss │
├──────────────────────┼───────┼─────────┼────────────┤
│ FastAI.LRFinderPhase │   1.0 │ 2.60303 │    5.87885 │
└──────────────────────┴───────┴─────────┴────────────┘


Learner()

In [8]:
using SimplePlots

In [10]:
SimplePlots.plot(phase.result.lrs, phase.result.losses, xscale=:log)

HTML{String}("  <div id=\"js-plot-e7d00ac9-206d-497d-b4c1-1e21d8b0d2c3\" style=\"width:600px;height:400px;\"></div>\n  <script class=\"js-new-plot-script\">\n    var anonFunc = function () {\n      plotDiv = document.getElementById('js-plot-e7d00ac9-206d-497d-b4c1-1e21d8b0d2c3');\n      if ( plotDiv === null ) { return; }\n      if ( plotDiv.childElementCount > 0 ) { return; }\n\n      Plotly.newPlot(\n        plotDiv,\n        [{\"line\":{\"color\":\"#636EFA\"},\"mode\":\"lines\",\"x\":[1.2022645e-7,1.4454398e-7,1.7378008e-7,2.089296e-7,2.5118865e-7,3.019952e-7,3.6307804e-7,4.3651582e-7,5.2480743e-7,6.3095735e-7,7.585776e-7,9.120108e-7,1.0964782e-6,1.3182567e-6,1.5848932e-6,1.9054607e-6,2.2908675e-6,2.7542287e-6,3.3113113e-6,3.9810716e-6,4.786301e-6,5.7543994e-6,6.9183097e-6,8.317637e-6,1.0e-5,1.2022644e-5,1.4454397e-5,1.7378008e-5,2.0892961e-5,2.5118865e-5,3.0199517e-5,3.6307807e-5,4.3651584e-5,5.2480747e-5,6.309574e-5,7.585776e-5,9.120109e-5,0.00010964782,0.00013182567,0.00015848932,0.00019054607,0.00022908677,0.00027542288,0.00033113113,0.00039810716,0.0004786301,0.00057543995,0.000691831,0.0008317638,0.001,0.0012022645,0.0014454398,0.0017378008,0.0020892962,0.0025118864,0.0030199517,0.0036307806,0.0043651583,0.0052480744,0.0063095735,0.0075857756,0.009120109,0.010964782,0.013182567,0.015848933,0.019054607,0.022908676,0.027542287,0.03311311,0.039810717,0.04786301,0.057543993,0.0691831,0.083176374,0.1,0.12022644,0.14454398,0.17378008,0.20892961,0.25118864,0.30199516,0.36307806,0.43651584,0.52480745,0.63095737],\"y\":[2.475003,2.555789,2.462898,2.6399715,2.5234284,2.4180195,2.5004377,2.4608266,2.6699762,2.504611,2.7341807,2.6546347,2.6470323,2.4623854,2.7735915,2.6449473,2.6625752,2.7160497,2.7218425,2.6796074,2.6082344,2.4382405,2.4595644,2.6533303,2.310246,2.7048056,2.70299,2.7658272,2.5197434,2.5487714,2.530415,2.7245884,2.565496,2.5580175,2.2614365,2.2302992,2.415085,2.4323137,2.5306692,2.260264,2.552186,1.9459507,2.337111,2.175266,2.0124664,2.215251,2.2232494,2.1819086,2.0182693,2.5298796,1.8372309,1.9576138,1.8246473,1.818549,2.471957,1.494344,2.140686,1.9853042,2.7924905,2.4439518,2.1440177,2.5542994,2.0610294,2.0784714,3.7377079,2.756812,2.607058,2.9744787,2.074322,2.9332151,2.7082107,2.3317802,4.652595,4.17605,4.267002,4.758175,2.4721932,2.663917,3.223466,2.277009,2.3393588,2.8054767,2.563726,2.8683825,6.142636]}],\n        {\"showlegend\":true,\"xaxis\":{\"autorange\":true,\"type\":\"log\",\"exponentformat\":\"power\"},\"annotations\":[],\"yaxis\":{\"autorange\":true,\"type\":\"linear\"},\"shapes\":[],\"legend\":{}},\n        {\"responsive\":true}\n      );\n    }\n\n    customPlotLoader(anonFunc);\n  </script>\n")